## Homework

In [84]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-29 18:22:50--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 142.250.141.102, 142.250.141.100, 142.250.141.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hi1hgsc31colq41ebs878i5vk23fhht0/1653848550000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-29 18:22:52--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hi1hgsc31colq41ebs878i5vk23fhht0/1653848550000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [85]:
import pandas as pd

In [86]:
df_orig = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [87]:
df_orig.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


In [88]:
df = df_orig.copy()

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [89]:
df.shape

(20000, 292)

In [90]:
df.isna().sum()[df.isna().sum() != 0]

life_sq                                  4103
floor                                     113
max_floor                                6303
material                                 6303
build_year                               8905
num_room                                 6303
kitch_sq                                 6303
state                                    8907
preschool_quota                          4279
school_quota                             4277
hospital_beds_raion                      9404
raion_build_count_with_material_info     3175
build_count_block                        3175
build_count_wood                         3175
build_count_frame                        3175
build_count_brick                        3175
build_count_monolith                     3175
build_count_panel                        3175
build_count_foam                         3175
build_count_slag                         3175
build_count_mix                          3175
raion_build_count_with_builddate_i

In [91]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [92]:
# Найс выброс
df.build_year[df.build_year > 2020] = 2007

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
df.build_year[df.build_year <= 1] = None

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
df = df.drop(drop_columns, axis=1)

In [95]:
df[num_columns] = df[num_columns].fillna(df[num_columns].mean())

In [96]:
df[cat_columns] = df[cat_columns].fillna(df[cat_columns].mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [97]:
df.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,1,1.0,1.0,1.000000,1.000000,1984.933397,1.00000,1.000000,1.000000,OwnerOccupier,...,36,7,2,15,33,1,12,75,10,15318960
1,64,64.0,16.0,12.603928,1.838505,1984.933397,1.91071,6.454917,2.106193,OwnerOccupier,...,2,2,0,0,13,1,0,6,1,6080000
2,83,44.0,9.0,17.000000,1.000000,1985.000000,3.00000,10.000000,3.000000,Investment,...,13,6,1,8,18,0,1,52,0,17000000
3,71,49.0,2.0,12.603928,1.838505,1984.933397,1.91071,6.454917,2.106193,Investment,...,0,0,0,1,3,0,2,8,2,990000
4,60,42.0,9.0,9.000000,1.000000,1970.000000,3.00000,6.000000,2.000000,Investment,...,3,1,0,5,8,0,1,34,5,7900000


In [98]:
df.replace('yes', 1, inplace=True)
df.replace('no', 0, inplace=True)

In [99]:
df[cat_columns]

,product_type,material,state,sub_area,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,...,ID_railroad_station_walk,ID_railroad_station_avto,water_1line,ID_big_road1,big_road1_1line,ID_big_road2,railroad_1line,ID_railroad_terminal,ID_bus_terminal,ecology
0,OwnerOccupier,1.000000,1.000000,Nagornoe,0,0,0,0,0,0,...,2.0,42,0,2,0,4,0,32,2,poor
1,OwnerOccupier,1.838505,2.106193,Poselenie Sosenskoe,0,0,0,0,0,0,...,47.0,105,0,38,0,1,0,32,8,no data
2,Investment,1.000000,3.000000,Krylatskoe,0,0,0,0,0,0,...,10.0,22,0,29,0,1,0,50,1,good
3,Investment,1.838505,2.106193,Matushkino,0,0,0,0,0,0,...,28.0,28,0,14,0,49,0,83,1,no data
4,Investment,1.000000,2.000000,Gol'janovo,0,0,0,0,1,0,...,18.0,136,1,1,0,12,0,97,7,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Investment,1.000000,2.000000,Jasenevo,0,0,0,0,1,0,...,42.0,73,1,1,0,43,0,32,8,good
19996,OwnerOccupier,1.000000,1.000000,Poselenie Sosenskoe,0,0,0,0,0,0,...,47.0,105,0,38,0,1,0,32,8,no data
19997,Investment,1.000000,2.000000,Moskvorech'e-Saburovo,0,0,0,0,1,0,...,29.0,27,0,2,0,4,0,32,2,excellent
19998,Investment,1.000000,2.106193,Poselenie Moskovskij,0,0,0,0,0,0,...,24.0,19,0,13,0,38,0,50,8,no data


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 290 entries, full_sq to price
dtypes: float64(119), int64(168), object(3)
memory usage: 44.3+ MB


In [101]:
# Кодируем категориальные признаки, их осталось не так много
df = pd.get_dummies(df, columns=['product_type', 'sub_area', 'ecology'])
df.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,sub_area_Vyhino-Zhulebino,sub_area_Zamoskvorech'e,sub_area_Zapadnoe Degunino,sub_area_Zjablikovo,sub_area_Zjuzino,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1,1.0,1.0,1.000000,1.000000,1984.933397,1.00000,1.000000,1.000000,5.293465e+06,...,0,0,0,0,0,0,0,0,1,0
1,64,64.0,16.0,12.603928,1.838505,1984.933397,1.91071,6.454917,2.106193,6.677245e+07,...,0,0,0,0,0,0,0,1,0,0
2,83,44.0,9.0,17.000000,1.000000,1985.000000,3.00000,10.000000,3.000000,1.216448e+07,...,0,0,0,0,0,0,1,0,0,0
3,71,49.0,2.0,12.603928,1.838505,1984.933397,1.91071,6.454917,2.106193,4.708040e+06,...,0,0,0,0,0,0,0,1,0,0
4,60,42.0,9.0,9.000000,1.000000,1970.000000,3.00000,6.000000,2.000000,1.428699e+07,...,0,0,0,0,0,0,1,0,0,0


In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1), df['price'], train_size=0.8)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [103]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

regr = DecisionTreeRegressor()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

Проверьте качество на отложенной выборке.

In [104]:
mean_squared_error(y_test, y_pred, squared=False)

3483624.570804026

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [105]:
month_year = (df_orig.timestamp.dt.month + df_orig.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df_orig.timestamp.dt.weekofyear + df_orig.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [106]:
df_orig['max_floor'] += 1

In [107]:
df['month'] = df_orig.timestamp.dt.month
df['weekday'] = df_orig.timestamp.dt.weekday
df['floor_per_maxfloor'] = df_orig['floor'] / df_orig['max_floor']
df['kitchen_per_flat'] = df_orig['kitch_sq'] / df_orig['full_sq']

In [108]:
df[['month', 'weekday', 'floor_per_maxfloor', 'kitchen_per_flat']] = df[['month', 'weekday', 'floor_per_maxfloor', 'kitchen_per_flat']].fillna(df[['month', 'weekday', 'floor_per_maxfloor', 'kitchen_per_flat']].mean())

In [110]:
df.describe()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory,month_year_cnt,week_year_cnt,month,weekday,floor_per_maxfloor,kitchen_per_flat
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,54.259950,34.032333,7.644441,12.603928,1.838505,1984.933397,1.910710,6.454917,2.106193,1.745676e+07,...,0.234750,0.246350,0.264950,0.124450,566.006400,138.439200,6.741600,2.201700,0.863193,0.131848
std,43.854961,18.129051,5.281414,5.542504,1.233619,24.966556,0.701437,25.121228,0.666309,2.053805e+07,...,0.423853,0.430896,0.441318,0.330102,249.950105,73.044868,3.510092,1.573131,1.421304,0.602166
min,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,1.000000,2.081628e+06,...,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,38.000000,22.000000,3.000000,9.000000,1.000000,1979.000000,1.910710,5.000000,2.000000,7.128794e+06,...,0.000000,0.000000,0.000000,0.000000,366.000000,83.000000,4.000000,1.000000,0.400000,0.104478
50%,49.000000,34.032333,7.000000,12.603928,1.838505,1984.933397,1.910710,6.454917,2.106193,1.020722e+07,...,0.000000,0.000000,0.000000,0.000000,550.000000,125.000000,6.000000,2.000000,0.777778,0.131848
75%,63.000000,38.000000,11.000000,16.000000,1.838505,1984.933397,2.000000,8.000000,2.106193,1.788191e+07,...,0.000000,0.000000,1.000000,0.000000,753.000000,177.250000,10.000000,3.000000,0.863193,0.153846
max,5326.000000,637.000000,44.000000,99.000000,6.000000,2018.000000,19.000000,2014.000000,33.000000,2.060718e+08,...,1.000000,1.000000,1.000000,1.000000,1122.000000,403.000000,12.000000,6.000000,25.000000,63.677419


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [121]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1), df.price, train_size=0.8)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [122]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

3598650.2895265524

In [124]:
df.price.mean()

7118798.6398

In [125]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=30, verbose=3)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 30


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


building tree 2 of 30


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.2s remaining:    0.0s


...

building tree 4 of 30


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   10.3s remaining:    0.0s


building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished


2554523.496337964

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [126]:
cat.fit(X_train, y_train)
y_pred = cat.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

Learning rate set to 0.06345
0:	learn: 4510435.9841047	total: 83.2ms	remaining: 1m 23s
1:	learn: 4377852.0242902	total: 145ms	remaining: 1m 12s
2:	learn: 4248287.9840978	total: 209ms	remaining: 1m 9s
3:	learn: 4134902.9718691	total: 269ms	remaining: 1m 6s
4:	learn: 4028336.7603642	total: 331ms	remaining: 1m 5s
5:	learn: 3933323.3114822	total: 397ms	remaining: 1m 5s
6:	learn: 3836718.2665229	total: 467ms	remaining: 1m 6s
7:	learn: 3748565.3151275	total: 529ms	remaining: 1m 5s
8:	learn: 3666932.6361881	total: 592ms	remaining: 1m 5s
9:	learn: 3597131.8504252	total: 653ms	remaining: 1m 4s
10:	learn: 3529697.1673010	total: 722ms	remaining: 1m 4s
11:	learn: 3464518.2082135	total: 788ms	remaining: 1m 4s
12:	learn: 3406469.1177187	total: 861ms	remaining: 1m 5s
13:	learn: 3356355.5940308	total: 928ms	remaining: 1m 5s
14:	learn: 3304310.6905877	total: 993ms	remaining: 1m 5s
15:	learn: 3257268.0348406	total: 1.06s	remaining: 1m 4s
16:	learn: 3211068.8686187	total: 1.12s	remaining: 1m 4s
17:	learn

2459274.387533567

RandomForest и Catboost показали себя лучше всего. Можно поподбирать гиперпараметры

In [133]:
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [136]:
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators': [10, 30],
    'min_samples_leaf': [1, 3],
    'max_depth': [None, 100]
}
grid = GridSearchCV(RandomForestRegressor(), params, verbose=10, scoring='neg_root_mean_squared_error', cv=2)
grid.fit(df.drop('price', axis=1), df.price)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2; 1/8] START max_depth=None, min_samples_leaf=1, n_estimators=10.........
[CV 1/2; 1/8] END max_depth=None, min_samples_leaf=1, n_estimators=10;, score=-2790045.525 total time=  17.4s
[CV 2/2; 1/8] START max_depth=None, min_samples_leaf=1, n_estimators=10.........
[CV 2/2; 1/8] END max_depth=None, min_samples_leaf=1, n_estimators=10;, score=-2961488.536 total time=  15.2s
[CV 1/2; 2/8] START max_depth=None, min_samples_leaf=1, n_estimators=30.........
[CV 1/2; 2/8] END max_depth=None, min_samples_leaf=1, n_estimators=30;, score=-2686348.665 total time=  46.0s
[CV 2/2; 2/8] START max_depth=None, min_samples_leaf=1, n_estimators=30.........
[CV 2/2; 2/8] END max_depth=None, min_samples_leaf=1, n_estimators=30;, score=-2834041.275 total time=  45.4s
[CV 1/2; 3/8] START max_depth=None, min_samples_leaf=3, n_estimators=10.........
[CV 1/2; 3/8] END max_depth=None, min_samples_leaf=3, n_estimators=10;, score=-2800808.423 tota

GridSearchCV(cv=2, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 100], 'min_samples_leaf': [1, 3],
                         'n_estimators': [10, 30]},
             scoring='neg_root_mean_squared_error', verbose=10)

In [137]:
grid.best_params_

{'max_depth': None, 'min_samples_leaf': 3, 'n_estimators': 30}

In [138]:
grid.best_score_

-2744215.1016903576

In [169]:
params = {
    'depth': [None, 5, 100]
}
grid = GridSearchCV(CatBoostRegressor(verbose=100), params, verbose=10, scoring='neg_root_mean_squared_error', cv=2)
grid.fit(df.drop('price', axis=1), df.price)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV 1/2; 1/3] START depth=None..................................................
Learning rate set to 0.058909
0:	learn: 4628505.4942593	total: 152ms	remaining: 2m 31s
100:	learn: 2378121.3212974	total: 6.18s	remaining: 55.1s
200:	learn: 2189688.9126678	total: 11.5s	remaining: 45.7s
300:	learn: 2039474.1259385	total: 16.8s	remaining: 39.1s
400:	learn: 1907009.1815139	total: 22.2s	remaining: 33.2s
500:	learn: 1802943.2941070	total: 27.7s	remaining: 27.5s
600:	learn: 1702626.9984782	total: 33.1s	remaining: 22s
700:	learn: 1622958.9410254	total: 38.6s	remaining: 16.4s
800:	learn: 1550867.9653349	total: 44s	remaining: 10.9s
900:	learn: 1482778.4912138	total: 49.4s	remaining: 5.43s
999:	learn: 1417196.6389946	total: 54.9s	remaining: 0us
[CV 1/2; 1/3] END ............depth=None;, score=-2497202.722 total time=  55.6s
[CV 2/2; 1/3] START depth=None..................................................
Learning rate set to 0.058909
0:	lear

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 6.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 5593, in fit
    save_snapshot, snapshot_file, snapshot_interval, init_model, callbacks, log_cout, log_cerr)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 2270, in _fit
    callbacks=callbacks
  File "/usr/local/lib/python3.7

Learning rate set to 0.065727
0:	learn: 4506831.1151283	total: 63.2ms	remaining: 1m 3s
100:	learn: 2469165.4507056	total: 4.52s	remaining: 40.2s
200:	learn: 2293977.7582019	total: 8.75s	remaining: 34.8s
300:	learn: 2176902.1484174	total: 13s	remaining: 30.1s
400:	learn: 2094541.9683725	total: 17.2s	remaining: 25.8s
500:	learn: 2029829.1091542	total: 21.5s	remaining: 21.4s
600:	learn: 1967345.3546293	total: 25.8s	remaining: 17.1s
700:	learn: 1912105.4079059	total: 30.2s	remaining: 12.9s
800:	learn: 1865648.8137173	total: 34.5s	remaining: 8.57s
900:	learn: 1821492.3036640	total: 38.8s	remaining: 4.26s
999:	learn: 1783056.4271977	total: 43.1s	remaining: 0us


GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f259df18650>,
             param_grid={'depth': [None, 5, 100]},
             scoring='neg_root_mean_squared_error', verbose=10)

In [170]:
grid.best_score_

-2557931.358435348

In [171]:
grid.best_params_

{'depth': 5}

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [181]:
invest_train_X = X_train[X_train['product_type_Investment'] == 1]
invest_train_y = y_train[X_train['product_type_Investment'] == 1]
owner_train_X = X_train[X_train['product_type_OwnerOccupier'] == 1]
owner_train_y = y_train[X_train['product_type_OwnerOccupier'] == 1]

In [182]:
model_invest = CatBoostRegressor(depth=5, verbose=100)
model_invest.fit(invest_train_X, invest_train_y)
y_pred = model_invest.predict(X_test[X_test['product_type_Investment'] == 1])
mean_squared_error(y_test[X_test['product_type_Investment'] == 1], y_pred, squared=False)

Learning rate set to 0.059154
0:	learn: 4772407.6197422	total: 46.3ms	remaining: 46.3s
100:	learn: 2754263.9038433	total: 3.74s	remaining: 33.3s
200:	learn: 2570740.6737489	total: 7.36s	remaining: 29.3s
300:	learn: 2434991.5747246	total: 11s	remaining: 25.5s
400:	learn: 2321756.1015208	total: 14.6s	remaining: 21.8s
500:	learn: 2225760.2806947	total: 18.2s	remaining: 18.2s
600:	learn: 2152617.0083267	total: 21.9s	remaining: 14.5s
700:	learn: 2084141.1198621	total: 25.5s	remaining: 10.9s
800:	learn: 2024400.4293305	total: 29.2s	remaining: 7.25s
900:	learn: 1973090.2235096	total: 32.8s	remaining: 3.6s
999:	learn: 1919917.5101880	total: 36.4s	remaining: 0us


2825704.402071533

In [179]:
model_owner = CatBoostRegressor(depth=5, verbose=100)
model_owner.fit(owner_train_X, owner_train_y)
y_pred = model_owner.predict(X_test[X_test['product_type_OwnerOccupier'] == 1])
mean_squared_error(y_test[X_test['product_type_OwnerOccupier'] == 1], y_pred, squared=False)

Learning rate set to 0.053953
0:	learn: 3953189.7584786	total: 64.5ms	remaining: 1m 4s
100:	learn: 1244297.9873295	total: 2.86s	remaining: 25.5s
200:	learn: 1055127.0794503	total: 5.39s	remaining: 21.4s
300:	learn: 919746.1664325	total: 7.91s	remaining: 18.4s
400:	learn: 831545.3690755	total: 10.4s	remaining: 15.6s
500:	learn: 770446.0359961	total: 13s	remaining: 12.9s
600:	learn: 710445.3529628	total: 15.5s	remaining: 10.3s
700:	learn: 669099.2357678	total: 18.1s	remaining: 7.7s
800:	learn: 624955.6906776	total: 20.6s	remaining: 5.12s
900:	learn: 587914.1819826	total: 23.2s	remaining: 2.55s
999:	learn: 560370.4394615	total: 25.7s	remaining: 0us


1524497.4693949549

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
# Не совсем понял задание(